In [27]:
#@title Download the data from website
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/movie_titles.csv
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_1.txt
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_2.txt
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_3.txt
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/netflix-data/combined_data_4.txt

from IPython.display import clear_output
clear_output()

print("Data in combined_data_1.txt looks like this: \n")
!head -5 combined_data_1.txt

print("\n\nData in movie_titles.csv looks like this: \n")

!head -5 movie_titles.csv

Data in combined_data_1.txt looks like this: 

1:
1488844,3,2005-09-06
822109,5,2005-05-13
885013,4,2005-10-19
30878,4,2005-12-26


Data in movie_titles.csv looks like this: 

1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW


In [28]:
f = open("movie_titles.csv", encoding ="cp1252")
data_lines = open("combined_data_1.txt", "r").readlines()
!wget -nc https://course.ccs.neu.edu/cs6220/fall2023/homework-1/basket_data.csv

--2023-09-18 06:52:53--  https://course.ccs.neu.edu/cs6220/fall2023/homework-1/basket_data.csv
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2357 (2.3K) [text/csv]
Saving to: ‘basket_data.csv’

basket_data.csv     100%[===================>]   2.30K  --.-KB/s    in 0s      

2023-09-18 06:52:53 (79.7 MB/s) - ‘basket_data.csv’ saved [2357/2357]



In [29]:
def cardinality_items( filename ):
  unique_items = set()
  with open(filename, 'r') as file:
    for line in file:
      items = [item.strip() for item in line.split(',')]
      unique_items.update(items)
  return len(unique_items)

res = cardinality_items("basket_data.csv")
print(res)

21


In [30]:
def all_itemsets(unique_items, N):
  if N == 0:
    return [set()]
  if N > len(unique_items):
    return []

  def generate_combinations(items, k):
    if k == 0:
      return [set()]
    if not items:
      return []

    include_first = [{items[0]} | subset for subset in generate_combinations(items[1:], k-1)]
    exclude_first = generate_combinations(items[1:], k)

    return include_first + exclude_first

  return generate_combinations(unique_items, N)


unique_items = ['ham', 'cheese', 'bread']
N = 2
print(all_itemsets(unique_items, N))


[{'ham', 'cheese'}, {'ham', 'bread'}, {'bread', 'cheese'}]


In [31]:
import os

def count_movie_ratings():
  total_ratings = 0

  files = os.listdir()
  print(files)

  combined_data_files = [f for f in files if f.startswith("combined_data_") and f.endswith(".txt")]

  for file_name in combined_data_files:
    with open(file_name, 'r') as file:
      for line in file:
        if not line.strip().endswith(':'):
          total_ratings += 1

  return total_ratings

print(count_movie_ratings())

['.config', 'combined_data_4.txt', 'movie_titles.csv', 'combined_data_1.txt', 'combined_data_3.txt', 'basket_data.csv', 'combined_data_2.txt', 'sample_data']
100480507


In [32]:
def count_unique_users():
  unique_users = set()

  files = os.listdir()

  combined_data_files = [f for f in files if f.startswith("combined_data_") and f.endswith(".txt")]

  for file_name in combined_data_files:
    with open(file_name, 'r') as file:
      for line in file:
        if not line.strip().endswith(':'):
          user_id = line.split(',')[0]
          unique_users.add(user_id)

  return len(unique_users)

print(count_unique_users())

480189


In [33]:
def find_date_range():
  earliest_year = float('inf')
  latest_year = float('-inf')

  files = os.listdir()

  combined_data_files = [f for f in files if f.startswith("combined_data_") and f.endswith(".txt")]

  for file_name in combined_data_files:
    with open(file_name, 'r') as file:
      for line in file:
        if not line.strip().endswith(':'):
          date = line.split(',')[-1].strip()
          year = int(date.split('-')[0])
          earliest_year = min(earliest_year, year)
          latest_year = max(latest_year, year)

  return (earliest_year, latest_year)

print(find_date_range())

(1999, 2005)


In [34]:
import csv
def count_unique_movie_names(filename):
  movie_names = set()

  with open(filename, 'r', encoding='cp1252') as file:
    reader = csv.reader(file)
    for row in reader:
      movie_name = row[2]
      movie_names.add(movie_name)

  return len(movie_names)

filename = "movie_titles.csv"
print(count_unique_movie_names(filename))


17297


In [35]:
def count_movie_names_with_four_references(filename):
  movie_name_counts = {}

  with open(filename, 'r', encoding='cp1252') as file:
    reader = csv.reader(file)
    for row in reader:
      movie_name = row[2]
      movie_name_counts[movie_name] = movie_name_counts.get(movie_name, 0) + 1

  count = sum(1 for name, occurrences in movie_name_counts.items() if occurrences == 4)
  return count

print(count_movie_names_with_four_references(filename))

7


In [36]:
def count_users_rated_200_movies():
  user_ratings_count = {}
  user_5_star_movies = {}

  files = os.listdir()

  combined_data_files = [f for f in files if f.startswith("combined_data_") and f.endswith(".txt")]

  current_movie_id = None
  for file_name in combined_data_files:
    with open(file_name, 'r') as file:
      for line in file:
        if line.strip().endswith(':'):
          current_movie_id = line[:-1]
        else:
          user_id, rating, _ = line.split(',')
          if rating == '5':
            if user_id not in user_5_star_movies:
              user_5_star_movies[user_id] = []
            user_5_star_movies[user_id].append(current_movie_id)
          user_ratings_count[user_id] = user_ratings_count.get(user_id, 0) + 1

  count = sum(1 for user, ratings in user_ratings_count.items() if ratings == 200)
  users_200_ratings = [user for user, count in user_ratings_count.items() if count == 200]
  return count, users_200_ratings, user_5_star_movies

count, users_200_ratings, user_5_star_movies = count_users_rated_200_movies()
print(count)
lowest_id_user = min(users_200_ratings, key=int)
print(lowest_id_user)
favorite_movie_ids = user_5_star_movies.get(lowest_id_user, [])


605
508


In [37]:
movie_names = {}
with open("movie_titles.csv", 'r', encoding='cp1252') as file:
  reader = csv.reader(file)
  for row in reader:
    movie_id, movie_name = row[0], row[2]
    movie_names[movie_id] = movie_name

print(len(favorite_movie_ids))

favorite_movie_names = []
for movie_id in favorite_movie_ids:
  movie_id = movie_id.split(':')[0]
  if movie_id in movie_names:
    favorite_movie_names.append(movie_names[movie_id])

print(favorite_movie_names)

57
['Raging Bull', 'Lord of the Rings: The Return of the King', 'Monty Python and the Holy Grail', 'Raising Arizona', 'The Shawshank Redemption: Special Edition', 'Harold and Maude', 'Downfall', 'Lord of the Rings: The Return of the King: Extended Edition', 'Monster', 'Band of Brothers', 'Three Kings', 'Unforgiven', 'Maria Full of Grace', 'Days of Wine and Roses', 'Shakespeare in Love', 'High Fidelity', "Monty Python's The Meaning of Life: Special Edition", 'American Beauty', 'Roger & Me', 'Eternal Sunshine of the Spotless Mind', 'Being John Malkovich', 'Vietnam: A Television History', 'Super Size Me', 'Lord of the Rings: The Fellowship of the Ring', 'This Is Spinal Tap', 'The Pianist', 'The Silence of the Lambs', 'Sideways', 'Whale Rider', 'Garden State', 'Bowling for Columbine', 'L.A. Confidential', 'Taxi Driver', 'Lord of the Rings: The Two Towers', 'Cabaret', 'Adaptation', 'The Accused', 'Lost in Translation', "Boys Don't Cry", 'To Be and To Have', "Schindler's List", 'Gandhi', 'Ap